### PTAL（墨田区）　データ作成手順書

---

#### 使用データ

1. 100mメッシュのポリゴンデータ（国土数値情報>土地利用細分メッシュ（地域5339、世界測地系、2021年度：https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-L03-b-2021.html#!））
2. バス停の位置情報（国土数値情報>バス停留所データ>東京都>2022年度））（墨田区の隣接地域も）
3. 駅の位置情報（国土数値情報>鉄道データ>2023年度）のインポート（墨田区の隣接地域も）
4. 町丁の境界線のポリゴンデータ（e-Stat>境界データダウンロード>小地域>国勢調査>2020年>町丁・字等）
5. 各SAP（駅・バス停）における、各路線の運行頻度に関する情報（時刻表）（インターネット上）

---

#### Step 0-1: SAPs (Service Access Points）とメッシュの中心点とを結ぶデータの作成（ArcGIS Proを使用）

* 上記1-4のデータのArcGISへのインポート 
* 町丁のポリゴンデータを市区町村名でフィルタリングして、墨田区のポリゴンだけを選択 → dissolveして墨田区の区界ポリゴンを作成（**Sumida_Border.geojson**）
* メッシュデータと墨田区の区界ポリゴンを使って、墨田区に含まれるメッシュを抽出（select by location → export）（intersect：境界線と一部でも交差するものは含める）（**Sumida_mesh.geojson**）
* 各メッシュの重心点のポイントデータを作成（feature to point）→ indexの値をコピーしてMeshIDという列を作る）（**Sumida_mesh_centroids.geojson**）
* 墨田区の区界の1km外側のポリゴンを作成（バッファーポリゴン）（SAPになりうるのは最も遠い点でも墨田区の区界から960m以内。不要なSAPを含めるとOD Cost Analysisで不必要なcredit消費をしてしまうため）
* バス停・駅のデータとバッファーポリゴンを使って、バッファー内に含まれるバス停・駅のポイントだけを抽出（select by location）（+indexの値をコピーしてそれぞれにBusStopID、StationIDという列を作る） 
* 駅はlineなのでfeature to pointでpointデータに変換 
* 墨田区内のメッシュの重心点とバッファー内のバス停・駅のポイントデータを使って、Make OD Cost Matrix Layerというtoolを使って、各重心点と徒歩距離で640m圏内にあるバス停と960m圏内にある駅をそれぞれ抽出し（4.8km/hでそれぞれ8分、12分）、それぞれの組み合わせにおける徒歩距離を算出（**mesh_to_busstops.geojason**と**mesh_to_stations.geojson**としてエクスポート）
* mesh_to_busstops.geojasonとバッファー内のバス停リストとをinner joinすることでMake OD Cost Matrix Layerに使用されたBusStopIDを抽出（駅も同様）（**Sumida_BusStops_Point.geojson**と**Sumida_Stations_Point.geojson**）


注：メッシュデータ、バス停や駅のデータを作った時に、MeshIDとStationID, BusStopIDという列を用意しておく事を忘れずに（インデックスとは別で用意しておく。
注：メッシュやバス停・駅の数が多く一括処理できない場合は、メッシュを分割して処理 

#### Step 0-2: 各SAPにおける運行頻度データの作成（インターネット上からexcelに手入力）

* バッファー内に含まれるバス停・駅のポイントデータと、OD Cost Matrixで作ったデータを使って、internal join（両方のデータが合致する行のみ残す）（バッファーポリゴンを使って切り出したデータは、必ずしも今回の分析対象となっていないため、実際に分析対象となったMake OD Cost Matrix Layerで残ったバス停・駅のリストを作成） 
* バス停・駅のリストのデータをArcGISからexcelファイルとしてexport 
* 各バス停のデータで1つのセルにまとまっている複数のバス路線のデータを、分解して、バス路線ごとに行を分ける → BusRoute IDの列を作成して、路線ごとに番号を割り当てる
* 駅データについては、総武線は元データでは1つだが、総武快速線と緩行線はルートが違うので、停車駅である錦糸町のデータを快速線と緩行線で分ける → Railway IDの列を作成して、路線ごとに番号を割り当てる
* TransitTypeという列を鉄道・バスのデータの両方に作り、鉄道のデータにはTの列、バスにはBの列を付ける　 
* 対象となる鉄道・バス路線の時刻表の収集・運行頻度（平日の午前8時15分から午前9時15分の間）を入力（両方向に運行されている場合は、より頻繁に運行されている方向を採用）し、**BusStops.csv**と**Stations.csv**としてエクスポート（データ作成に使ったexcelファイルは**ServiceFrequencyCalc**というフォルダ内の**PTAL_TransitFq_Sumida.xlsx**）
    - 墨田区内の区間で真ん中付近にありかつ始発ではない駅を選択
    - 快速・普通などの運行区分がある場合には、路線全体で1つの運行頻度を適応しないように。快速停車駅かどうかの確認をして、各停しか止まらない駅のデータをそのまま快速停車駅に適応しないように。
    - 各路線の運行頻度の基準とした駅・バス停と基準とした方向をexcelに記録
    - 路線の一部だけ1方向のバス停がある。’NumberOfDirections’という列を作り、1方向運行（0）か、2方向運行（1）かを入力（PTALには含まれていないが、後々使う事があるかもしれないので念のため）

注：MeshIDとStationID, BusStopIDと同様にバス路線・鉄道路線についてもインデックスとは別でBusRoute ID、RailwayID、というのを作っておく

---

#### Step 0-3: データの読み込み

In [1]:
import geopandas as gpd
import pandas as pd

#墨田区の100m四方のメッシュポリゴン(shp)の読み込み
esh = gpd.read_file("data/Sumida_mesh.geojson")

#各メッシュの中心点と各SAP(service access points：徒歩640m圏内にあるバス停、徒歩960m圏内にある駅）を結んだshpの読み込み
mesh_busstop = gpd.read_file("data/mesh_to_busstops.geojson")
mesh_station = gpd.read_file("data/mesh_to_stations.geojson")

#各バス停における各バスルートの運行頻度データ(csv)の読み込み
busstop = pd.read_csv("data/BusStops.csv")
station = pd.read_csv("data/Stations.csv")

#### Step 1: 各SAPへの歩行時間の計算

In [2]:
#メッシュの中心点とSAPを結ぶデータには所要時間も入っている（'Total_Walk'）が、
#5km/hの歩行速度で計算されたものなので、
#距離データ（'Total_Kilo'）を使って、4.8km/hの歩行速度での所要時間に変換

mesh_busstop['WalkTime'] = mesh_busstop['Total_Kilometers']/4.8*60
mesh_station['WalkTime'] = mesh_station['Total_Kilometers']/4.8*60

#### Step 2: 各SAPの予定待ち時間（SWT）の算出

In [3]:
#SWT(scheduled waiting time) = 0.5 * (60/頻度)
busstop['SWT'] = 0.5*(60/ busstop['PeakHourFq'])
station['SWT'] = 0.5*(60/ station['PeakHourFq'])

#### Step 3: 各SAPの平均待ち時間（AWT）の算出

In [4]:
#AWT(average waiting times)=SWT + 信頼係数
#バスには信頼性係数2分、鉄道、地下鉄、路面電車には信頼性係数0.75分
busstop['AWT'] = busstop['SWT']+2
station['AWT'] = station['SWT']+0.75

#### Step 4: 各SAPの各ルートの合計アクセス時間（TAT）の算出

In [5]:
joined_mesh_busstop = pd.merge(mesh_busstop, busstop, on='BusStopID', how='left', suffixes=('', '_from_busstop'))
joined_mesh_station = pd.merge(mesh_station, station, on='StationID', how='left', suffixes=('', '_from_station'))
#joinをする（many to many）：左にgdf、右にcsv、BusStopIDとStationIDをキーに
#joinの前後の行の数を確認
#やってる操作が合っているかどうかの再確認を

In [6]:
#TAT(total access time) = 移動時間（'WalkTime'）+AWT
joined_mesh_busstop['TAT']=joined_mesh_busstop['WalkTime']+joined_mesh_busstop['AWT']
joined_mesh_station['TAT']=joined_mesh_station['WalkTime']+joined_mesh_station['AWT']

In [7]:
joined_mesh_busstop.columns

Index(['OBJECTID', 'Name', 'OriginID', 'DestinationID', 'DestinationRank',
       'Total_WalkTime', 'Total_Kilometers', 'Name_tentative',
       'OriginID_tentative', 'Shape_Length', 'MeshID', 'BusStopID', 'geometry',
       'WalkTime', 'BusStopName', 'BusRouteName', 'BusRouteID',
       'NumberOfDirections', 'PeakHourFq', 'PeakHourFq_less', 'RefBusStop',
       'RefDirection', 'DailyFq', 'TransitType', 'SWT', 'AWT', 'TAT'],
      dtype='object')

In [8]:
joined_mesh_busstop

,OBJECTID,Name,OriginID,DestinationID,DestinationRank,Total_WalkTime,Total_Kilometers,Name_tentative,OriginID_tentative,Shape_Length,...,NumberOfDirections,PeakHourFq,PeakHourFq_less,RefBusStop,RefDirection,DailyFq,TransitType,SWT,AWT,TAT
0,1,Location 1 - Location 414,1,414,1,2.512106,0.209342,None,NaN,0.001543,...,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN
1,1,Location 1 - Location 414,1,414,1,2.512106,0.209342,None,NaN,0.001543,...,2.0,2.0,2.0,錦糸町駅前,築地駅前,NaN,B,15.0,17.0,19.616776
2,1,Location 1 - Location 414,1,414,1,2.512106,0.209342,None,NaN,0.001543,...,2.0,5.0,4.0,本所吾妻橋,亀戸駅前,NaN,B,6.0,8.0,10.616776
3,2,Location 1 - Location 415,1,415,2,2.512106,0.209342,None,NaN,0.001543,...,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN
4,3,Location 1 - Location 398,1,398,3,3.022972,0.251920,None,NaN,0.002312,...,1.0,5.0,NaN,両国駅西口,NaN,NaN,B,6.0,8.0,11.148997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22866,13161,None,1441,15,2,6.900283,0.575011,Location 441 - Location 15,441.0,0.007422,...,2.0,5.0,5.0,堀切菖蒲園駅,新小岩駅,NaN,B,6.0,8.0,15.187633
22867,13161,None,1441,15,2,6.900283,0.575011,Location 441 - Location 15,441.0,0.007422,...,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN
22868,13162,None,1445,16,1,6.334307,0.527829,Location 445 - Location 16,445.0,0.006084,...,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN
22869,13163,None,1445,15,2,7.015231,0.584569,Location 445 - Location 15,445.0,0.007290,...,2.0,5.0,5.0,堀切菖蒲園駅,新小岩駅,NaN,B,6.0,8.0,15.307115


#### Step5: 各SAPのEDFの算出

In [9]:
#EDF(equivalent doorstep frequency) = 0.5 * (60 / TAT)
#EDFはTATを時間ではなく頻度の単位に変換したもの
joined_mesh_busstop['EDF']=0.5*(60/joined_mesh_busstop['TAT'])
joined_mesh_station['EDF']=0.5*(60/joined_mesh_station['TAT'])

#### Step6: アクセス指数（AI）の算出

In [10]:
#AI（Access Index）とは、
#各MeshIDについて、同じMeshIDをもつ行の中でEDFが最も高いものに1の重みを与え、同じMeshIDをもつ行の中で他のすべてのEDFには0.5の重みを与える
#その上で、各MeshIDごとにそれらを合計（sum）したものである
#1つのバッファー内に同じ路線のバス停があった場合の処理の仕方を確認（最も近いものを採用？そういう処理はchatGPTで可能？）

#### バスのAIを計算

In [11]:
#ステップ1: 各 MeshID ごとに EDF の最大値を見つける
joined_mesh_busstop['max_edf'] = joined_mesh_busstop.groupby('MeshID')['EDF'].transform('max')

# ステップ2: 重み列を作成（最大EDFには1、それ以外は0.5）
joined_mesh_busstop['weight'] = joined_mesh_busstop.apply(lambda row: 1 if row['EDF'] == row['max_edf'] else 0.5, axis=1)

# ステップ3: MeshID ごとに重みを合計（= Access Index）
AI_bus = joined_mesh_busstop.groupby('MeshID')['weight'].sum().reset_index(name='AI')

In [12]:
print(len(AI_bus))
print(mesh_busstop['MeshID'].nunique())
print(len(esh))

1391
1391
1445


In [13]:
# 各セットを取得
ai_set = set(AI_bus['MeshID'])
esh_set = set(esh['MeshID'])

# AIにはあるがeshにはないMeshID
only_in_ai = ai_set - esh_set

# eshにはあるがAIにはないMeshID
only_in_esh = esh_set - ai_set

print("AIにしかないMeshID:", only_in_ai)
print("eshにしかないMeshID:", only_in_esh)

AIにしかないMeshID: set()
eshにしかないMeshID: {1152, 1408, 1409, 1400, 1287, 1288, 1415, 1416, 1417, 1421, 1422, 1423, 1427, 1428, 1429, 1435, 1436, 1439, 1440, 1442, 1443, 1444, 1407, 1089, 1098, 1099, 1363, 1108, 1109, 1364, 1117, 1118, 1119, 1373, 1374, 1381, 1126, 1127, 1128, 1129, 1382, 1383, 1390, 1391, 1392, 1139, 1143, 1144, 1401, 1147, 1148, 1399, 1150, 1151}


In [14]:
#この差の54個のmeshIDはすべて北東部の川沿いであることが判明
#MeshIDは以下のすべて
#1089, 1098, 1099, 1108, 1109, 1117, 1118, 1119, 1126, 1127, 1128, 1129, 1139, 1143, 1144, 1147, 1148, 1150, 1151, 1152, 1287, 1288, 1363, 1364, 1373, 1374, 1381, 1382, 1383, 1390, 1391, 1392, 1399, 1400, 1401, 1407, 1408, 1409, 1415, 1416, 1417, 1421, 1422, 1423, 1427, 1428, 1429, 1435, 1436, 1439, 1440, 1442, 1443, 1444
#これらの行についてはAIが出てこないことに留意

In [15]:
AI_bus['AI'].isnull().sum()
#busstopという運行頻度データにはデータが入っているが、null valueの路線も存在する
#そのためMeshIDによっては、運行の無いバス路線としか紐づいていないものも存在するかと懸念したが、それは無かった

0

#### 鉄道のAIを計算

In [16]:
#ステップ1: 各 MeshID ごとに EDF の最大値を見つける
joined_mesh_station['max_edf'] = joined_mesh_station.groupby('MeshID')['EDF'].transform('max')

# ステップ2: 重み列を作成（最大EDFには1、それ以外は0.5）
joined_mesh_station['weight'] = joined_mesh_station.apply(lambda row: 1 if row['EDF'] == row['max_edf'] else 0.5, axis=1)

# ステップ3: MeshID ごとに重みを合計（= Access Index）
AI_train = joined_mesh_station.groupby('MeshID')['weight'].sum().reset_index(name='AI')

In [17]:
print(len(AI_train))
print(mesh_station['MeshID'].nunique())
print(len(esh))

1311
1311
1445


In [18]:
# 各セットを取得
ai_set = set(AI_train['MeshID'])
esh_set = set(esh['MeshID'])

# AIにはあるがeshにはないMeshID
only_in_ai = ai_set - esh_set

# eshにはあるがAIにはないMeshID
only_in_esh = esh_set - ai_set

print("AIにしかないMeshID:", only_in_ai)
print("eshにしかないMeshID:", only_in_esh)

AIにしかないMeshID: set()
eshにしかないMeshID: {1036, 1037, 1038, 1039, 1044, 1045, 1046, 1047, 1048, 1049, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1076, 1077, 1078, 1079, 1087, 1088, 1089, 1097, 1098, 1099, 1108, 1109, 1117, 1118, 1119, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1154, 1155, 1157, 1158, 1160, 1161, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1250, 262, 263, 1288, 1343, 1344, 1354, 1363, 1364, 861, 1373, 1374, 1382, 1383, 875, 1392, 881, 882, 373, 374, 375, 376, 1401, 892, 893, 894, 382, 385, 386, 898, 899, 903, 396, 397, 912, 916, 1430, 407, 920, 1433, 923, 1435, 1436, 926, 1438, 929, 493}


In [19]:
#この差の134個のmeshIDは北西部・北部・東部の川沿いと南部の中央部であることが判明
#MeshIDは以下のすべて
#262, 263, 373, 374, 375, 376, 382, 385, 386, 396, 397, 407, 493, 861, 875, 881, 882, 892, 893, 894, 898, 899, 903, 912, 916, 920, 923, 926, 929, 1036, 1037, 1038, 1039, 1044, 1045, 1046, 1047, 1048, 1049, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1076, 1077, 1078, 1079, 1087, 1088, 1089, 1097, 1098, 1099, 1108, 1109, 1117, 1118, 1119, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1154, 1155, 1157, 1158, 1160, 1161, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1250, 1288, 1343, 1344, 1354, 1363, 1364, 1373, 1374, 1382, 1383, 1392, 1401, 1430, 1433, 1435, 1436, 1438
#これらの行についてはAIが出てこないことに留意

#### メッシュのポリゴンデータとバス・鉄道それぞれのAIの値を結合

In [20]:
#AccessIndexという列はAI_busにもAI_trainにもあるので、重複しないようにAccessIndex_bus、AccessIndex_trainと列名を変える
AI_bus = AI_bus.rename(columns={'AI': 'AI_bus'})
AI_train = AI_train.rename(columns={'AI': 'AI_train'})

#eshとAI_bus、AI_trainにはそれぞれMeshIDという列が存在する
#その列をキーにして、eshのgeodataframeにAI_busとAI_trainのdataframeを結合
#まずeshにAI_busを左結合
esh_merged = esh.merge(AI_bus, on='MeshID', how='left')
#さらにAI_trainを左結合
esh_merged = esh_merged.merge(AI_train, on='MeshID', how='left')

In [26]:
#esh_mergedというgeodataframeのAI_busとAI_trainの列の値について、null valueのものは0に置き換える
esh_merged['AI_bus'] = esh_merged['AI_bus'].fillna(0)
esh_merged['AI_train'] = esh_merged['AI_train'].fillna(0)

In [27]:
#最終的なAccessIndexを算出
esh_merged['AI']=esh_merged['AI_bus']+esh_merged['AI_train']

In [29]:
esh_merged['AI'].isnull().sum()

0

#### 総合のAIをマッピング

In [35]:
esh_merged = esh_merged.to_crs(epsg=4326)

import folium
import branca.colormap as cm

# 地図の中心を計算（平均緯度・経度）
centroid = esh_merged.geometry.centroid
center_lat = centroid.y.mean()
center_lon = centroid.x.mean()

# Foliumマップの作成
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# AI列の最大最小
min_val = esh_merged['AI'].min()
max_val = esh_merged['AI'].max()

# カラーマップを定義
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)
colormap.caption = 'Access Index (AI)'

# GeoJsonでマップに描画
folium.GeoJson(
    esh_merged,
    name="AI Layer",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['AI']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['MeshID', 'AI'])
).add_to(m)

# カラーマップをマップに追加
colormap.add_to(m)

#htmlファイルとして出力
m.save("AI_map.html")

# マップを表示
m

/var/folders/lx/_3z8wwyx3ds15_0k28hby_h80000gn/T/ipykernel_53043/3831130226.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = esh_merged.geometry.centroid


#### 鉄道のAIをマッピング

In [33]:
# 地図の中心座標を計算（平均緯度・経度）
centroid = esh_merged.geometry.centroid
center_lat = centroid.y.mean()
center_lon = centroid.x.mean()

# Foliumマップの作成
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# 値の範囲を取得
min_val = esh_merged['AI_train'].min()
max_val = esh_merged['AI_train'].max()

# カラーマップを作成
colormap = cm.linear.YlGnBu_09.scale(min_val, max_val)
colormap.caption = 'Access Index - Train (AI_train)'

# GeoJsonで可視化
folium.GeoJson(
    esh_merged,
    name="AI_train Layer",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['AI_train']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(fields=['MeshID', 'AI_train'])
).add_to(m)

# カラーマップを追加
colormap.add_to(m)

# マップを表示
m

/var/folders/lx/_3z8wwyx3ds15_0k28hby_h80000gn/T/ipykernel_53043/3343728945.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = esh_merged.geometry.centroid


#### Step 7: PTALへの変換

#### Mapping

In [23]:
#html化させる？

#### データの保存

In [24]:
joined_mesh_busstop.to_file("joined_mesh_busstop.gpkg", driver="GPKG")

#### 留意点
- 運行種別
    - 新宿線や伊勢崎線、浅草線には快速と普通など運行種類の違いがある。同じ方向でも異なる行き先の場合もある。
    - ただ快速停車駅と普通のみ停車駅とでは運行頻度が違うので、路線で一律に運行頻度を適応せず、駅の種別で異なる時刻表を適用
    - 行き先の違いについては考慮しない。
    - 錦糸町駅は総武快速線と総武各駅停車線が両方停まる唯一の駅。それらを別のルートとして処理するかどうか。
        - 国土数値情報では別の路線としては処理されておらず、総武線という1つとしての扱い。ただ実際にはルートが異なるので、厳密には別路線として処理する事とする。

- 運行の方向性
    - PTALのマニュアルでは1方向運行か2方向運行かは加味しない
    - 今回は、後々方向性を加味して分析したくなる可能性を考慮して、1方向運行は1、2方向運行は2として’NumberOfDirections’という列にデータを入れておく
    - 始発のバス停・駅は’NumberOfDirections’を1を割り当て
    - 錦11など、一部区間だけ1方向のバス停がある事例がある
        - 路線図・時刻表を確認の上で、1方向のバス停には’NumberOfDirections’を1を割り当て
    - 上記以外の双方向の出発があるバス停には2を割り当て
    - 押上駅は構造が複雑で、4路線が乗り入れている。各路線の起点駅になっており、路線別で分けるとすべて1方向運行だが、実際には直通運転しているので2方向運行の実態。今回は1方向運行の4路線という処理にする（最もアクセス性の高い路線だけに1を掛ける、という重み付けの時に、路線を統合した場合と別々に処理した場合とで差が出るかもしれない）

- PTALでは、近いバス停は統合しているが鉄道駅についての言及はない
    - なので、鉄道駅についてまとめる必要はないが、今回は、国土数値情報が出している300m以内かつ同名の駅を対象としたグループ分けがあるので、そのデータは’StationIDGrouped’という列として残しておく

#### ロンドンのPTALの枠組みを墨田で適応する上で考慮しなければならない要素
- **歩行速度**（高齢者、東京）：
    - アクセスポイントまでの設定距離を短くするならば、Step1においてmesh_to_busstops.geojsonとmesh_to_stations.geojsonのTotal_Kilometersや、計算に使う歩行速度を変更すればよい
    - アクセスポイントまでの設定距離を長くするならばArcGisProでのMake OD Cost Matrix Layerの操作設定を変更してデータを作り直す必要がある
- **遅延（信頼性係数）**（東京）：
    - Step3における係数を調整
- **採用する時間帯**
    - ラッシュアワーの時間帯（東京）
    - 高齢者の移動時間帯がラッシュアワーかどうか）（高齢者）
        - いずれもStep0-2（各SAPにおける運行頻度データの作成）においてデータを作り直す必要がある
- **バスと鉄道の重要性の比率**（東京）